In [85]:
import econml
import numpy as np
import urllib
import os

import pandas as pd
import matplotlib.pyplot as plt

# Multiple continuous treatment synthetic data

In [4]:
np.random.seed(123)
n = 6000
n_w = 30
support_size = 5
n_x = 5
# Outcome support
support_Y = np.random.choice(np.arange(n_w), size=support_size, replace=False)
coefs_Y = np.random.uniform(0, 1, size=support_size)
epsilon_sample = lambda n: np.random.uniform(-1, 1, size=n)
# Treatment support
support_T = support_Y
coefs_T = np.random.uniform(0, 1, size=support_size)
eta_sample = lambda n: np.random.uniform(-1, 1, size=n)

# Generate controls, covariates, treatments and outcomes
W = np.random.normal(0, 1, size=(n, n_w))
X = np.random.uniform(0, 1, size=(n, n_x))
# Heterogeneous treatment effects
TE1 = np.array([x_i[0] for x_i in X])
TE2 = np.array([x_i[0]**2 for x_i in X]).flatten()
T = np.dot(W[:, support_T], coefs_T) + eta_sample(n)
Y = TE1 * T + TE2 * T**2 + np.dot(W[:, support_Y], coefs_Y) + epsilon_sample(n)
# Generate test data
X_test = np.random.uniform(0, 1, size=(100, n_x))
X_test[:, 0] = np.linspace(0, 1, 100)

In [5]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.preprocessing import PolynomialFeatures

from econml.dml import LinearDML

est = LinearDML(model_y=GradientBoostingRegressor(n_estimators=100, max_depth=3, min_samples_leaf=20),
                model_t=MultiOutputRegressor(GradientBoostingRegressor(n_estimators=100,
                                                                       max_depth=3,
                                                                       min_samples_leaf=20)),
                featurizer=PolynomialFeatures(degree=2, include_bias=False),
                linear_first_stages=False,
                cv=5)

In [6]:
T = T.reshape(-1, 1)
est.fit(Y, np.concatenate((T, T**2), axis=1), X=X, W=W)

In [7]:
te_pred = est.const_marginal_effect(X_test)

In [9]:
te_pred

array([[-1.47605603e-01, -5.90702594e-02],
       [ 3.25825946e-02, -3.64446981e-02],
       [ 1.17251522e-01, -1.29122793e-02],
       [ 1.28267838e-01, -8.61959784e-02],
       [-2.52532118e-02, -6.24744413e-02],
       [ 1.94987552e-02, -8.64571380e-02],
       [-2.92381054e-02, -2.82841605e-02],
       [ 1.13254665e-01, -6.87775701e-02],
       [ 7.06807551e-02, -4.40644038e-02],
       [ 1.89476975e-02, -2.34866150e-02],
       [ 9.76090052e-02, -3.33281046e-02],
       [ 9.75819515e-02, -3.97002024e-02],
       [ 7.54787563e-02, -3.05907647e-02],
       [ 1.90444057e-01, -5.12901697e-02],
       [ 1.82476821e-01, -2.16247440e-02],
       [ 1.49923173e-01,  1.51990289e-03],
       [ 4.28532551e-02, -5.97058968e-02],
       [ 3.36212777e-01,  1.31814129e-04],
       [ 1.93759888e-01,  9.42724434e-03],
       [ 2.72257105e-01,  1.87765430e-02],
       [ 2.84486621e-01, -5.25171979e-03],
       [ 6.26061096e-02, -1.01353094e-02],
       [ 2.37953174e-01,  4.23498062e-03],
       [ 2.

# OJ

https://rdrr.io/cran/bayesm/man/orangeJuice.html

In [13]:
# Import the data
file_name = "oj_large.csv"

if not os.path.isfile(file_name):
    print("Downloading file (this might take a few seconds)...")
    urllib.request.urlretrieve("https://msalicedatapublic.z5.web.core.windows.net/datasets/OrangeJuice/oj_large.csv", file_name)
oj_data = pd.read_csv(file_name)

In [16]:
oj_data.head()

,store,brand,week,logmove,feat,price,AGE60,EDUC,ETHNIC,INCOME,HHLARGE,WORKWOM,HVAL150,SSTRDIST,SSTRVOL,CPDIST5,CPWVOL5
0,2,tropicana,40,9.018695,0,1.353255,0.232865,0.248935,0.11428,10.553205,0.103953,0.303585,0.463887,2.110122,1.142857,1.92728,0.376927
1,2,tropicana,46,8.723231,0,1.353255,0.232865,0.248935,0.11428,10.553205,0.103953,0.303585,0.463887,2.110122,1.142857,1.92728,0.376927
2,2,tropicana,47,8.253228,0,1.353255,0.232865,0.248935,0.11428,10.553205,0.103953,0.303585,0.463887,2.110122,1.142857,1.92728,0.376927
3,2,tropicana,48,8.987197,0,1.353255,0.232865,0.248935,0.11428,10.553205,0.103953,0.303585,0.463887,2.110122,1.142857,1.92728,0.376927
4,2,tropicana,50,9.093357,0,1.353255,0.232865,0.248935,0.11428,10.553205,0.103953,0.303585,0.463887,2.110122,1.142857,1.92728,0.376927


In [36]:
from sklearn.preprocessing import StandardScaler

oj_data = pd.read_csv(file_name)
oj_data["price"] = np.log(oj_data["price"])

groupbylist = ["store", "week", "AGE60", "EDUC", "ETHNIC", "INCOME",
               "HHLARGE", "WORKWOM", "HVAL150",
               "SSTRDIST", "SSTRVOL", "CPDIST5", "CPWVOL5"]
oj_data1 = pd.pivot_table(oj_data,index=groupbylist,
                          columns=oj_data.groupby(groupbylist).cumcount(),
                          values=['logmove', 'price'],
                          aggfunc='sum').reset_index()

oj_data1.columns = oj_data1.columns.map('{0[0]}{0[1]}'.format) 
oj_data1 = oj_data1.rename(index=str,
                           columns={"logmove0": "logmove_T",
                                    "logmove1": "logmove_M",
                                    "logmove2":"logmove_D",
                                    "price0":"price_T",
                                    "price1":"price_M",
                                    "price2":"price_D"})

# Define Y,T,X,W
Y = oj_data1['logmove_T'].values
T = oj_data1[['price_T', "price_M", "price_D"]].values
scaler = StandardScaler()
W=scaler.fit_transform(oj_data1[[c for c in groupbylist if c not in ['week', 'store', 'INCOME']]].values)
X=scaler.fit_transform(oj_data1[['INCOME']].values)


In [117]:
from sklearn.linear_model import MultiTaskElasticNetCV, LassoCV
from econml.dml import DML

est = DML(model_y=GradientBoostingRegressor(),
         model_t=MultiTaskElasticNetCV(cv=3),
        model_final=LassoCV()
                #featurizer=PolynomialFeatures(),
                #linear_first_stages=True
                )
est.fit(Y, T, X=X, W=W, inference="bootstrap")

min_income = -1
max_income = 1
delta = (1 - (-1)) / 100
X_test = np.arange(min_income, max_income + delta - 0.001, delta).reshape(-1, 1)

te_pred = est.const_marginal_effect(X_test)


The final model has a nonzero intercept for at least one outcome; it will be subtracted, but consider fitting a model without an intercept if possible.
The final model has a nonzero intercept for at least one outcome; it will be subtracted, but consider fitting a model without an intercept if possible.
The final model has a nonzero intercept for at least one outcome; it will be subtracted, but consider fitting a model without an intercept if possible.
The final model has a nonzero intercept for at least one outcome; it will be subtracted, but consider fitting a model without an intercept if possible.
The final model has a nonzero intercept for at least one outcome; it will be subtracted, but consider fitting a model without an intercept if possible.
The final model has a nonzero intercept for at least one outcome; it will be subtracted, but consider fitting a model without an intercept if possible.
The final model has a nonzero intercept for at least one outcome; it will be subtracted,

In [119]:
est.summary()

,point_estimate,stderr,zstat,pvalue,ci_lower,ci_upper
X0|T0,0.183,0.027,6.676,0.0,0.121,0.233
X0|T1,-0.002,0.021,-0.089,0.48,-0.037,0.045
X0|T2,-0.039,0.023,-1.71,0.08,-0.078,0.011
,point_estimate,stderr,zstat,pvalue,ci_lower,ci_upper
cate_intercept|T0,-2.958,0.03,-99.968,0.0,-3.034,-2.92
cate_intercept|T1,0.361,0.026,13.775,0.0,0.318,0.414
cate_intercept|T2,0.202,0.021,9.725,0.0,0.166,0.24


In [123]:
from econml.cate_interpreter import SingleTreePolicyInterpreter

intrp = SingleTreePolicyInterpreter(risk_level=None, max_depth=2, min_samples_leaf=1, min_impurity_decrease=.001)
intrp.interpret(est, X, sample_treatment_costs=1)
intrp.plot()
plt.savefig("tree_inpt.png")

In [97]:
est.summary()

,point_estimate,stderr,zstat,pvalue,ci_lower,ci_upper
1|T0,-1.513,0.02,-77.279,0.0,-1.551,-1.475
1|T1,0.196,0.016,11.939,0.0,0.164,0.228
1|T2,0.11,0.013,8.667,0.0,0.085,0.135
X0|T0,0.221,0.029,7.511,0.0,0.163,0.279
X0|T1,-0.017,0.025,-0.697,0.486,-0.066,0.032
X0|T2,-0.05,0.019,-2.629,0.009,-0.088,-0.013
X0^2|T0,0.051,0.021,2.459,0.014,0.01,0.092
X0^2|T1,-0.012,0.017,-0.674,0.501,-0.046,0.022
X0^2|T2,-0.011,0.014,-0.812,0.417,-0.038,0.016
,point_estimate,stderr,zstat,pvalue,ci_lower,ci_upper


In [128]:
import dowhy
model 

econml has not been tested with dowhy versions >= 0.9


AssertionError: Can only accept single dimensional treatment.